# Leveraging Sparse Observations to Predict Species Abundance Across Space and Time

This note book shows the step by step execution of the supporting codes in this repository. Please see `readme.md` for more details. 

In [7]:
import data.dataprocessing as dp
from configs import data_config as dc

data_obj = dp.GBRDataset(fileloc=dc.rawdata_loc,
                          poissonmodel=True,
                          fdim=2,                          
                          normalize=False,
                          savefeatures=True
                          )


****************************************
Executing data processing......
Parameters: 
	Poisson:True
	verbose: True
	poissonmodel: True
	fdim: 2
	normalize: False
	savefeatures: True
	data file location: data/rawdata/gbr_cots_culldata.xlsx

****************************************
Executing data processing......
Reading data file from: data/rawdata/gbr_cots_culldata.xlsx
Panda DataFrame shape:(4427, 9).
Getting the sites, site features and locations....
1368 sites
Got 1368 sites, (1368, 2) site features and 1368 locations.
Parsing the data....
earliest: 1
latest: 817
max gap without between two voyage: 1 days
817 days of data
Got a list of lists. length of data is 817.
Computing the distance matrix....
Distance metrix D with shape (1368, 1368).
Normalizing the static features....
Shape of normalized static features: (1368, 2).
Building the feature matrix....
Shape of the feature matrix: (817, 1368, 2).


## Expected output

- The output shown here are using the data used in the paper. Note that we removed indentificaiton informaiton form the supplied data. 
```
****************************************
Executing data processing......
Parameters: 
	Poisson:True
	verbose: False
	poissonmodel: True
	fdim: 2
	normalize: False
	savefeatures: True
	data file location: data/rawdata/gbr_culldata.xlsx

****************************************
Executing data processing......
Reading data file from: data/rawdata/gbr_culldata.xlsx
Panda DataFrame shape:(4427, 9).
Getting the sites, site features and locations....
Got 1368 sites, (1368, 2) site features and 1368 locations.
Parsing the data....
Got a list of lists. length of data is 983.
Computing the distance matrix....
Distance metrix D with shape (1368, 1368).
Normalizing the static features....
Shape of normalized static features: (1368, 2).
Building the feature matrix....
Shape of the feature matrix: (983, 1368, 2).
```

In [8]:
import numpy as np

print("===============")
print("Dataset statics")
print("===============")
data_obj.print_statistics()

sid = 100
tid = 300
for item1, item2 in zip(data_obj.X[tid+1, :, 0], data_obj.Y[tid, :]):
    if item1 > 0 or item2 >0:
        print(item1, item2)
    print((data_obj.X[tid+1, :, 0]==data_obj.Y[tid, :]).sum())
    print(np.array_equal(data_obj.X[tid + 1, :, 0],
                         data_obj.Y[tid, :]
                         )
          )

    print("\n\n")
    for item1, item2 in zip(data_obj[tid+1]['x'][0], data_obj[tid]['y']):
        if item1 > 0 or item2 > 0:
            print(item1, item2)
    print(data_obj[tid+1]['x'], data_obj[tid]['y'])

    for ran in [100, 200, 300, 400, 500]:
        print(f'Front end sparsity: range({ran})')
        frontend = data_obj.X[:ran, :, 0]
        print(f"frontend shape: {frontend.shape}")
        print(f"number of observed cell: {len(frontend[frontend>-1])}, "
              f"ratio: {len(frontend[frontend>-1])/(frontend.shape[0]*frontend.shape[1]):0.4f}")

        print('Backend sparsity:')
        backend = data_obj.X[-ran:, :, 0]
        print(f"backend shape: {backend.shape}")
        print(f"number of observed cell: {len(backend[backend > -1])}, "
              f"ratio: {len(backend[backend > -1]) / (backend.shape[0] * backend.shape[1]):0.4f}")
        print('**'*30)
        break

Dataset statics
Date time: 2024-09-07 18:48:18.438964
Average 5.4 voyages per day
Maxmium 23 voyages per day
Number of sites: 1368
Number of days-span: 816
minimum #cots: 0
maximum #cots: 3141
average #cots: 62.6
#cots < 1000: 0.995
#cots < 100: 0.841
#cots < 10: 0.358
every 14 days:
    repeat visit ratio: 0.39
    mininum 36 culling voyages
    average 76.5 culling voyages
0 days do not have data
sparsity: 0.0040
1368
True



23.0 -1.0
[[ -1.  23.]
 [ -1. 815.]
 [ -1. 815.]
 ...
 [ -1. 815.]
 [ -1. 815.]
 [ -1. 815.]] [-1. -1. -1. ... -1. -1. -1.]
Front end sparsity: range(100)
frontend shape: (100, 1368)
number of observed cell: 463, ratio: 0.0034
Backend sparsity:
backend shape: (100, 1368)
number of observed cell: 608, ratio: 0.0044
************************************************************
1368
True



23.0 -1.0
[[ -1.  23.]
 [ -1. 815.]
 [ -1. 815.]
 ...
 [ -1. 815.]
 [ -1. 815.]
 [ -1. 815.]] [-1. -1. -1. ... -1. -1. -1.]
Front end sparsity: range(100)
frontend shape: (100, 1

## Divide the data into train/test splits

In [ ]:
import numpy as np

gbrdata = np.load(r"data/processed-data/feature_mat_dim_2_norm_False_20240905.npz")
gbrdata.files

['X', 'Y']

In [3]:
import data.datautils as dutil

datadic = dutil.get_train_test_valid(gbrdata, testratio=.25, valratio=0., validation=False )
xtrain, ytrain = datadic['trainX'], datadic['trainY']
xtest, ytest = datadic['testX'], datadic['testY']

In [4]:
xtrain.shape, xtest.shape

((612, 1368, 2), (204, 1368, 2))

## Run the baselines

This reports the MAE using the following baselines:
- Global average
- Site average
- last seen

In [5]:
import model.baselines as bl
#from model.loss_fns import zMAE, zRMSE

baselines = bl.BaselineModels(X_train=xtrain, X_test=xtest, fdim=2, varbose=False)

gmae, gzmae, gzrmse = baselines.model_global_average()
sav_mae, sav_zmae, sav_zrmse = baselines.model_site_average()
lseen_mae, lseen_zmae, lseen_zrmse = baselines.model_last_seen()

print(f"Global average MAE: {gmae:.2f}, zMAE: {gzmae:.2f}, zRMSE: {gzrmse:.2f}")
print(f"Site average MAE: {sav_mae:.2f}, zMAE: {sav_zmae:.2f}, zRMSE: {sav_zrmse:.2f}")
print(f"Last seen average MAE: {lseen_mae:.2f}, zMAE: {lseen_zmae:.2f}, zRMSE: {lseen_zrmse:.2f}")

Global average MAE: 60.18, zMAE: 37.37, zRMSE: 54.62
Site average MAE: 51.29, zMAE: 28.06, zRMSE: 53.05
Last seen average MAE: 50.52, zMAE: 31.65, zRMSE: 53.17


## Model construction 

In [9]:
fdim = 2
poissonmodel = True
kernelwidth = 1400
threshold = 1e-5
model_name = "GCN"

modelparams = dict(hidden_dims=[128, 64, 64, 256],
                   fdim=fdim,
                   poissonmodel=poissonmodel,
                   nsites=xtrain.shape[1],
                   kernelwidth=kernelwidth,
                   threshold=threshold,
                   D=data_obj.get_distmat(),
                   model_name=model_name,
                   device="cpu",
                   site_fdim=2,
                   ndays=xtrain.shape[0],
                )

modelparams.keys()

dict_keys(['hidden_dims', 'fdim', 'poissonmodel', 'nsites', 'kernelwidth', 'threshold', 'D', 'model_name', 'device', 'site_fdim', 'ndays'])

In [10]:
import model.gbrmodel as mdl

model = mdl.GBRModel(**modelparams)
model

GBRModel(
  (inp_encoder): InputEncoder(
    (input_layers): Sequential(
      (0): Linear(in_features=2, out_features=128, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): ELU(alpha=1.0)
    )
  )
  (temp_encoder): TemporalEncoder(
    (temporal_layer): GRU(128, 128, batch_first=True)
  )
  (sp_encoder): SpatialEncoder(
    (Z_transform): Sequential(
      (0): Linear(in_features=2, out_features=128, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (gc_encoder): GCNEncoder(
    (gcn_layer1): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (output_encoder): OutputEncoder(
    (output_layers): Sequential(
      (0): ELU(alpha=1.0)
      (1): Linear(in_features=128, out_features=64, bias=True)
      (2): ELU(alpha=1.0)
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ELU(alpha=1.0)
      (5): Linear(

## Model training

In [14]:
import model.loss_fns as lfn
import torch
import torch.optim as optim
import trainer as tr
from tqdm.auto import tqdm
import copy

#initalize weights
model.apply(model.init_weights)

nepochs = 100
lr = 0.000001
momentum = 0.9
batchsize = 20

In [9]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# initialize the trainer
trainer = tr.Trainer(model, model_name, data_obj.Z, optimizer, (xtrain, ytrain), (xtest, ytest), batchsize)

best_mae_loss, best_weights = np.inf, None
# train the model
for i in tqdm(range(nepochs), total=nepochs, desc="Training: "):
    train_result = trainer.train_epoch()
    test_result = trainer.valid_epoch()
    print(f"EPOCH: {i} train_loss:{train_result['loss']:.5f}, eval_loss:{test_result['vloss']:.5f}")

    if test_result["vmae"] < best_mae_loss:
        best_mae_loss = test_result["vmae"]
        best_weights = copy.deepcopy(trainer.get_model().state_dict())
        print(f"Model saved at epoch: {i}")


Training:   0%|          | 0/100 [00:00<?, ?it/s]

EPOCH: 0 train_loss:296.72168, eval_loss:193.39554
Model saved at epoch: 0
EPOCH: 1 train_loss:290.27151, eval_loss:188.21056
Model saved at epoch: 1
EPOCH: 2 train_loss:283.32776, eval_loss:183.00813
Model saved at epoch: 2
EPOCH: 3 train_loss:276.38333, eval_loss:177.82262
Model saved at epoch: 3
EPOCH: 4 train_loss:269.45670, eval_loss:172.65549
Model saved at epoch: 4
EPOCH: 5 train_loss:262.54816, eval_loss:167.50706
Model saved at epoch: 5
EPOCH: 6 train_loss:255.65741, eval_loss:162.37761
Model saved at epoch: 6
EPOCH: 7 train_loss:248.78394, eval_loss:157.26741
Model saved at epoch: 7
EPOCH: 8 train_loss:241.92711, eval_loss:152.17656
Model saved at epoch: 8
EPOCH: 9 train_loss:235.08592, eval_loss:147.10506
Model saved at epoch: 9
EPOCH: 10 train_loss:228.25917, eval_loss:142.05270
Model saved at epoch: 10
EPOCH: 11 train_loss:221.44514, eval_loss:137.01913
Model saved at epoch: 11
EPOCH: 12 train_loss:214.64166, eval_loss:132.00366
Model saved at epoch: 12
EPOCH: 13 train_los

In [10]:
# reload the latest model
model.load_state_dict(best_weights)

<All keys matched successfully>

In [13]:
# save model weights
torch.save(model.state_dict(), "artifacts/best_model.mdl")

In [15]:
# load model
#model = model = mdl.GBRModel(**modelparams)
model.load_state_dict(torch.load("artifacts/best_model.mdl"))

C:\Users\isl020\AppData\Local\Temp\ipykernel_22744\274058824.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("artifacts/best_model.mdl")

<All keys matched successfully>

## Model evaluation

In [19]:
# need to build the context matrix
X_all = np.vstack([xtrain, xtest])
X = torch.tensor(X_all[:], dtype=torch.float32)

model.eval()
model.temp_encoder.reset_context()

"""
divide the data into batches
"""
num_batches = int(np.ceil(X.shape[0] / batchsize))
        
with torch.no_grad():
    predictions = []
    for i in range(num_batches):
        X_batch = X[i*batchsize: (i + 1)*batchsize]
        Y_batch, A = model(X_batch, data_obj.Z)

        predictions.append(Y_batch.detach())

N_test = xtest.shape[0]
Y_pred = torch.vstack(predictions)[-N_test:]
Y_true = torch.tensor(ytest, dtype=torch.float32)

count = torch.sum((Y_true >= 0).float())
loss = lfn.zip_loss(Y_pred, Y_true) / count

In [20]:
Y_pred.shape, Y_true.shape

(torch.Size([204, 1368, 2]), torch.Size([204, 1368]))

In [22]:
zmae = lfn.zMAE(Y_pred.detach().numpy(), Y_true.detach().numpy())
zrmse = lfn.zRMSE(Y_pred.detach().numpy(), Y_true.detach().numpy())
zmae, zrmse

(24.430852216416756, 45.00488651261445)